# 

# Gen3 Schema Development Framework
- This notebook provides a framework for developing a gen3 data schema using gdocs as a template.

### Pulling data schema from google sheets
- pulls design from the following google sheet template [link](https://docs.google.com/spreadsheets/d/1zjDBDvXgb0ydswFBwy47r2c8V1TFnpUj1jcG0xsY7ZI/edit?usp=sharing)
- This step involves pulling the schema design from a Google Sheet template. The template can be accessed [here](https://docs.google.com/spreadsheets/d/1zjDBDvXgb0ydswFBwy47r2c8V1TFnpUj1jcG0xsY7ZI/edit?usp=sharing). Feel free to duplicate this spreadsheet and input your own google sheet id along with the tab ids for objects, links, properties, and enums.

In [ ]:
import os
if os.path.basename(os.getcwd()) == 'jupyter':
    os.chdir('..')
os.getcwd()

In [ ]:
!cd umccr-dictionary && make down
!cd umccr-dictionary && make pull
!cd umccr-dictionary && make up
!cd umccr-dictionary && make ps

In [3]:
# Purging old google sheets
!test -d output/input_google_sheets && rm -r output/input_google_sheets

In [ ]:
# Pulling data schema from google sheets
# Note this is loads a gdoc template 
!rm -R schema_out
# pull google sheets and save to ./schema_out
!python3 sheet2yaml-CLI.py --google-id '1zjDBDvXgb0ydswFBwy47r2c8V1TFnpUj1jcG0xsY7ZI' --objects-gid 0 --links-gid 270346573 --properties-gid 613332252 --enums-gid 1807456496  --download --output-dir 'output/input_google_sheets'

In [ ]:
# Moving schema_out to umccr-dictionary
!mkdir -p umccr-dictionary/dictionary/schema_dev/gdcdictionary/schemas
!cp schema_out/* umccr-dictionary/dictionary/schema_dev/gdcdictionary/schemas/
!ls -lsha umccr-dictionary/dictionary/schema_dev/gdcdictionary/schemas/

In [ ]:
# compiling and bundling into json
!cd umccr-dictionary && make compile program=schema_dev


In [ ]:
# Running Validation
!cd umccr-dictionary && make validate program=schema_dev

In [8]:
# Visualising data dictionary
!open http://localhost:8080/#schema/schema_dev.json

In [9]:
# Purging old schema
!test -d output/schema && rm -r output/schema
!test -d output/schema && rm -r output/schema

In [10]:
# Copying outputs to root
!mkdir -p output/schema/yaml
!mkdir -p output/schema/json
!cp schema_out/* output/schema/yaml/
!cp umccr-dictionary/schema/schema_dev.json output/schema/json/

In [ ]:
# Resolving Schema
import json
from gen3schemadev.gen3schematools import ResolveSchema

resolver = ResolveSchema("output/schema/json/schema_dev.json")

with open("output/schema/json/schema_dev_resolved.json", "w") as f:
    json.dump(resolver.schema_resolved, f, indent=4)